In [1]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install tenacity

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install openai[embeddings]

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [6]:
import openai

In [7]:
from dotenv import dotenv_values

In [8]:
config = dotenv_values(".env")

In [9]:
openai.api_key = config["OPENAI_API_KEY"]

In [10]:
response = openai.embeddings.create(
 input="Your text string goes here",
 model="text-embedding-3-small"
)

In [11]:
print(response.data[0].embedding)

[0.0051719858311116695, 0.017202626913785934, -0.018700333312153816, -0.018560361117124557, -0.047310721129179, -0.03031805530190468, 0.027672573924064636, 0.0036707802210003138, 0.011232797056436539, 0.006424739956855774, -0.001675296458415687, 0.01583089493215084, -0.0013157420326024294, -0.007866457104682922, 0.05990825220942497, 0.050306133925914764, -0.027504606172442436, 0.009889060631394386, -0.040396079421043396, 0.049998193979263306, -0.00041007582331076264, 0.030262066051363945, -0.013731307350099087, 0.03292154520750046, 0.0173146054148674, 0.01679670624434948, -0.0017811506986618042, 0.020421994850039482, 0.04078800231218338, -0.03770860657095909, -0.026146871969103813, -0.05002618953585625, 0.024187257513403893, -0.05526116490364075, -0.03230566531419754, 0.04235569387674332, 0.06466732174158096, 0.014655126258730888, -0.015690922737121582, -0.041347891092300415, 0.02218564972281456, 0.007355557754635811, 0.044959183782339096, 0.007107106503099203, -0.024103272706270218, 0

In [12]:
dataset_path = "./movie_plots.csv"

In [13]:
pip install tiktoken

Note: you may need to restart the kernel to use updated packages.


In [14]:
import pandas as pd
import numpy as np
import pickle
import tiktoken

In [15]:
df = pd.read_csv(dataset_path);

In [16]:
movies = df[df["Origin/Ethnicity"]=="Bollywood"].sort_values("Release Year", ascending=False).head(1000)

In [17]:
movies

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
27127,2017,Tiger Zinda Hai,Bollywood,Ali Abbas Zafar,Salman Khan & Katrina Kaif,action/drama,https://en.wikipedia.org/wiki/Tiger_Zinda_Hai,"During a dark night in Iraq, an American journ..."
27075,2017,A Death in the Gunj,Bollywood,Konkona Sen Sharma,"Vikrant Massey, Ranvir Shorey, Kalki Koechlin,...",crime/drama,https://en.wikipedia.org/wiki/A_Death_in_the_Gunj,"Set in 1979, A Death in the Gunj is a coming o..."
27067,2017,Begum Jaan,Bollywood,Srijit Mukherji,"Vidya Balan, Ila Arun, Gauhar Khan, Sumit Nijh...",period drama,https://en.wikipedia.org/wiki/Begum_Jaan,"In August 1947, after India sought independenc..."
27068,2017,Noor,Bollywood,Sunhil Sippy,"Sonakshi Sinha, Purab Kohli & Manish Chaudhary",comedy/drama,https://en.wikipedia.org/wiki/Noor_(film),"Noor is a young journalist, battling to find h..."
27069,2017,Ek Thi Rani Aisi Bhi,Bollywood,Gulbahar Singh,"Hema Malini, Vinod Khanna, Sachin Khedekar",biopic,https://en.wikipedia.org/wiki/Ek_Thi_Rani_Aisi...,Ek Thi Rani Aisi Bhi is a story of conflict be...
...,...,...,...,...,...,...,...,...
26160,2002,Yeh Hai Jalwa,Bollywood,David Dhawan,"Rishi Kapoor, Salman Khan, Amisha Patel, Rinke...",comedy,https://en.wikipedia.org/wiki/Yeh_Hai_Jalwa,The story is about an orphan named Raju (Salma...
26159,2002,Waah! Tera Kya Kehna,Bollywood,Unknown,"Govinda, Raveena Tandon, Preeti Jhangiani",unknown,https://en.wikipedia.org/wiki/Waah!_Tera_Kya_K...,Wealthy Kishan Oberoi (Shammi Kapoor) has two ...
26158,2002,Tumse Achcha Kaun Hai,Bollywood,Unknown,"Rati Agnihotri, Aarti Chabria, Ali Asghar",unknown,https://en.wikipedia.org/wiki/Tumse_Achcha_Kau...,Working as a guide and bus driver in Jaisalmer...
26157,2002,Tumko Na Bhool Paayenge,Bollywood,Unknown,"Salman Khan, Dia Mirza, Sushmita Sen",drama,https://en.wikipedia.org/wiki/Tumko_Na_Bhool_P...,Veer Thakur (Salman Khan) is an eligible young...


In [18]:
from tenacity import retry, wait_random_exponential, stop_after_attempt

In [19]:
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def get_embedding(text, model="text-embedding-3-small"):
    text = text.replace("/n", " ");
    return openai.embeddings.create(
         input=text,
         model=model
    ).data[0].embedding

In [20]:
#This is path for movie embeddings
embedding_cache_path = "movie_embeddings.pkl"

In [21]:
try:
    embedding_cache = pd.read_pickle(embedding_cache_path)
except FileNotFoundError:
    embedding_cache = {}
    # Python code to write file using with 
with open(embedding_cache_path, "wb") as embedding_cache_file:
        pickle.dump(embedding_cache, embedding_cache_file)
        

def embedding_from_string(string, model = "text-embedding-3-small", embedding_cache = embedding_cache):
    if(string, model) not in embedding_cache.keys():
        embedding_cache[(string,model)] = get_embedding(string, model)
        print(f"GOT EMBEDDING FROM OPEN AI FOR STRING {string[:20]}")
        with open(embedding_cache_path, "wb") as embedding_cache_file:
            pickle.dump(embedding_cache, embedding_cache_file)
    return embedding_cache[(string, model)]
    

In [22]:
movie_plots = movies["Plot"].values

In [23]:
enc = tiktoken.encoding_for_model("text-embedding-3-small")
enc.encode("Hello world")

[9906, 1917]

In [24]:
 total_tokens = sum([len(enc.encode(plot)) for plot in movie_plots])
    

In [25]:
total_tokens

593774

In [26]:
cost = (0.00002 * total_tokens) / 1000

In [27]:
print("Estimated cost is : $",cost)

Estimated cost is : $ 0.01187548


In [28]:
plot_embeddings = [embedding_from_string(plot, model = "text-embedding-3-small") for plot in movie_plots]

In [29]:
len(plot_embeddings)

1000

In [36]:
pip install scipy

In [37]:
from scipy import spatial

def distances_from_embeddings(
    query_embedding: list,
    embeddings: list,
    distance_metric="cosine",
) -> list:
    """Return the distances between a query embedding and a list of embeddings."""
    distance_metrics = {
        "cosine": spatial.distance.cosine,
        "L1": spatial.distance.cityblock,
        "L2": spatial.distance.euclidean,
        "Linf": spatial.distance.chebyshev,
    }
    distances = [
        distance_metrics[distance_metric](query_embedding, embedding)
        for embedding in embeddings
    ]
    return distances

def indices_of_nearest_neighbors_from_distances(distances) -> np.ndarray:
    """Return a list of indices of nearest neighbors from a list of distances."""
    return np.argsort(distances)


In [56]:
def print_recommendatons_from_strings(strings, index_of_source_string, k_nearest_neighbours=3, model="text-embedding-3-small", movies=movies):
    embeddings = [embedding_from_string(string) for string in strings]
    query_embedding = embeddings[index_of_source_string]
    distances = distances_from_embeddings(query_embedding, embeddings)
    indices_of_nearest_neighbors = indices_of_nearest_neighbors_from_distances(distances)
    query_string = strings[index_of_source_string]
    count = 0
    
    for i in indices_of_nearest_neighbors:
        if(strings[i] == query_string):
            continue
        if count >= 2:
            break
        count += 1
        print("::::::::::::::NEW MATCH :::::::::::::::::::")
        print("Found closest match: ",)
        print("The distance is ", distances[i])
        print(strings[i])
    print(indices_of_nearest_neighbors)
    

In [57]:
print_recommendatons_from_strings(movie_plots, 0)

::::::::::::::NEW MATCH :::::::::::::::::::
Found closest match: 
The distance is  0.1722775726497663
An agent code-named "Tiger" (Salman Khan) – India's top spy and an officer of the Research and Analysis Wing (RAW) – is conducting a mission in northern Iraq. Tiger is forced to kill one of his own men who has defected to Pakistan's spy agency Inter-Services Intelligence (ISI). He subsequently kills many ISI agents in his escape from Iraq. Tiger returns to India and reports to his boss, Shenoy (Girish Karnad), in New Delhi.
Tiger is immediately sent on a mission to Dublin to observe a scientist of Indian origin, Professor Anwar Jamaal Kidwai (Roshan Seth), who teaches at Trinity College and is suspected of sharing his findings with the Pakistan defence establishment. Fellow RAW agent Gopi (Ranvir Shorey) accompanies Tiger on this mission.
Tiger meets the scientist but fails to spend enough time for his observance and instead begins to spend time with the scientist's caretaker Zoya (Kat

In [66]:
import os
os.path.abspath("")

'C:\\Users\\91963\\Untitled Folder'

27127    action/drama
27075     crime/drama
27067    period drama
27068    comedy/drama
27069          biopic
             ...     
26160          comedy
26159         unknown
26158         unknown
26157           drama
26156         musical
Name: Genre, Length: 1000, dtype: object